In [1]:
import pandas as pd
import numpy as np
import csv
import uuid

In [2]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/skos/SKOS_EVENTS.csv"

skos_event = pd.read_csv(PATH,header=0)
display(skos_event)

,skos:Concept,rdf:type,skos:inScheme,skos:broader,skos:prefLabel@fr,skos:prefLabel@en,skos:altLabel@fr,skos:altLabel@en,skos:hiddenLabel,skos:definition@fr
0,EventMentionnedinFoliosColumns,Event,CadastreEventList,NaN,Evènement mentionné dans les colonnes dédiées ...,NaN,NaN,NaN,NaN,Evènement mentionné explicitement dans les col...
1,TaxpayerMutation,Event,CadastreEventList,NaN,Mutation de propriétaire,NaN,NaN,NaN,NaN,NaN
2,Sale,Event,CadastreEventList,TaxpayerMutation,Vente,NaN,NaN,NaN,NaN,NaN
3,Sucession,Event,CadastreEventList,TaxpayerMutation,Sucession,NaN,NaN,NaN,NaN,NaN
4,PlotNatureEvent,Event,CadastreEventList,NaN,Evènement impliquant la nature d'une parcelle,NaN,NaN,NaN,NaN,Evènement impliquant la nature de la parcelle ...
5,ChangeWithoutConstruction,Event,CadastreEventList,PlotNatureEvent,Evolution de nature n'impliquant pas de bâtiment,NaN,NaN,NaN,NaN,NaN
6,Construction,Event,CadastreEventList,PlotNatureChange,Construction d'un bâtiment,NaN,NaN,NaN,NaN,NaN
7,Demotion,Event,CadastreEventList,PlotNatureChange,Démolition d'un bâtiment,NaN,NaN,NaN,NaN,NaN
8,BuiltPlotEvent,Event,CadastreEventList,PlotNatureChange,Evènement impliquant un bâtiment existant,NaN,NaN,NaN,NaN,NaN
9,Addition,Event,CadastreEventList,BuiltPlotEvent,Augmentation,NaN,NaN,NaN,NaN,NaN


In [3]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastre/eventType/")
g.bind('cad_etype', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
g.bind('cad', cad)
g.bind('add', add)

# Iterate over each row in the DataFrame
for index, row in skos_event.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, add.Event))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:altLabel@fr']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:prefLabel@en']):
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if not pd.isnull(row['skos:altLabel@en']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if not pd.isnull(row['skos:broader']):
        g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if not pd.isnull(row['skos:definition@fr']):
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition@fr"],lang='fr')))

    if not pd.isnull(row['skos:hiddenLabel']):
        if ',' in row["skos:hiddenLabel"]:
            for hiddenLabel in row["skos:hiddenLabel"].split(','):
                g.add((subject_uri, SKOS.hiddenLabel, Literal(hiddenLabel)))
        else:
            g.add((subject_uri, SKOS.hiddenLabel, Literal(row["skos:hiddenLabel"])))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

cad_etype:Addition a add:Event,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader cad_etype:BuiltPlotEvent ;
    skos:inScheme cad:CadastreEventList ;
    skos:prefLabel "Augmentation"@fr .

cad_etype:CadastreCreation a add:Event,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader cad_etype:CadastreAdminEvent ;
    skos:definition "Création d'un cadastre (= une numérotation)"@fr ;
    skos:inScheme cad:CadastreEventList ;
    skos:prefLabel "Initialisation d'un cadastre"@fr .

cad_etype:CadastreOutdated a add:Event,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader cad_etype:CadastreAdminEvent ;
    skos:definition "Archivage d'un cadastre"@fr ;
  